**ID3 Algorithm**

**Importing libraries and dataset**

In [0]:
import numpy as np
import pandas as pd
eps = np.finfo(float).eps
from numpy import log2 as log

In [0]:
df = pd.read_csv('/content/ID3_data.csv')

In [3]:
df

,outlook,temp,humidity,windy,play
0,overcast,hot,high,False,yes
1,overcast,cool,normal,True,yes
2,overcast,mild,high,True,yes
3,overcast,hot,normal,False,yes
4,rainy,mild,high,False,yes
5,rainy,cool,normal,False,yes
6,rainy,cool,normal,True,no
7,rainy,mild,normal,False,yes
8,rainy,mild,high,True,no
9,sunny,hot,high,False,no


Finding entropy of data D,     \\
Info(D) = $-\sum_{i=1}^{m}p_ilog_2(p_i)$  \\
Where $p_i$ is the probability that an atribute tuple in D belongs to class $C_i$ and \\
 is estimated by $\dfrac{\mid C_{i,D} \mid}{\mid D \mid}$

In [0]:
def find_entropy(df):
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy
  

Entropy of data D for given attribute A,  \\
$Info_A(D) = \sum_{i=1}^{v} \dfrac{\mid D_j \mid}{\mid D \mid} Info(D_j)$   \\
$\dfrac{\mid D_j \mid}{\mid D \mid}$ is weight of the $j^{th}$ partition.

In [0]:
def find_entropy_attribute(df,attribute):
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    target_variables = df[Class].unique()  #This gives all 'Yes' and 'No'
    variables = df[attribute].unique()    #This gives different features in that attribute (like 'Hot','Cold' in Temperature)
    entropy2 = 0
    for variable in variables:
        entropy = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df[Class] ==target_variable])
            den = len(df[attribute][df[attribute]==variable])
            fraction = num/(den+eps)
            entropy += -fraction*log(fraction+eps)
        fraction2 = den/len(df)
        entropy2 += -fraction2*entropy
    return abs(entropy2)



Finding the best attribute that has highest information gain.

In [0]:
def find_winner(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:
#         Entropy_att.append(find_entropy_attribute(df,key))
        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
    return df.keys()[:-1][np.argmax(IG)]


In [0]:
def get_subtable(df, node,value):
    return df[df[node] == value].reset_index(drop=True)


Creating tree

In [0]:
def buildTree(df,tree=None): 
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    
    #Here we build our decision tree

    #Get attribute with maximum information gain
    node = find_winner(df)
    
    #Get distinct value of that attribute e.g Salary is node and Low,Med and High are values
    attValue = np.unique(df[node])
    
    #Create an empty dictionary to create tree    
    if tree is None:                    
        tree={}
        tree[node] = {}
    
   #We make loop to construct a tree by calling this function recursively. 
    #In this we check if the subset is pure and stops if it is pure. 

    for value in attValue:
        
        subtable = get_subtable(df,node,value)
       # print(subtable)
        clValue,counts = np.unique(subtable[Class],return_counts=True)                        
        
        if len(counts)==1:#Checking purity of subset
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable) #Calling the function recursively 
                   
    return tree

In [9]:
t = buildTree(df)
import pprint

pprint.pprint(t)

{'outlook': {'overcast': 'yes',
             'rainy': {'windy': {False: 'yes', True: 'no'}},
             'sunny': {'humidity': {'high': 'no', 'normal': 'yes'}}}}


**Implementation of CART Algorithm**

**Importing numpy library, and datasets from sklearn**

In [0]:
import numpy as np
from sklearn import datasets

**Creating node class**

In [0]:
class Node:
    def __init__(self, pred_class):
        self.pred_class = pred_class        # define the predicted class for node
        self.left = None                              # initialize the left child of node
        self.right = None                             # initialize the left child of node
        self.feat_index = 0                        # initialize feature index of node
        self.threshold = 0                            # initialize threshold of node
        

Foe given data D, impurity is measured by Gini index. \\
$Gini(D) = 1 - \sum_{i=1}^{m}p_i^2$   \\

For discrete-valued attribute A, impurity of D is,  \\
$Gini_A(D) =  \dfrac{\mid D_1 \mid}{\mid D \mid}Gini(D_1) + \dfrac{\mid D_2 \mid}{\mid D \mid}Gini(D_2)$  \\
where $D_1$ and $D_2$ are splitted data.


For a discrete-valued attribute, the subset that gives the minimum gini index for that attribute is selected as its splitting subset. Which i have done in **best_split** method.

In [0]:
class DTC:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth                # define how much depth the tree has

    def fit(self, X, y):
        self.n_classes = len(set(y))             # To get the number of distinct class from the data
        self.n_features = X.shape[1]             # To get number of features in data
        self.tree = self.make_tree(X, y)        # Make a tree

    def make_tree(self, X, y, depth=0):
        # finding number of samples of each class 
        samples_per_class = [np.sum(y == i) for i in range(self.n_classes)]         
        # Get the class which has maximum number of samples
        pred_class = np.argmax(samples_per_class)
        # Creating node of class which has maximum number of samples
        node = Node(pred_class=pred_class)

        if depth < self.max_depth:
            index, thr = self.best_split(X, y)     # get best feature index and threshold for given data
            if index is not None:                   # if feature index exist
                left_indices = X[:, index] < thr    # index whose data has less than threshold value
                # splitting data in left and right
                left_X, left_y = X[left_indices], y[left_indices]
                right_X, right_y = X[~left_indices], y[~left_indices]
                node.feat_index = index
                node.threshold = thr
                # making further node in splitted data
                node.left = self.make_tree(left_X, left_y, depth + 1)
                node.right = self.make_tree(right_X, right_y, depth + 1)
        return node

    def best_split(self, X, y):
        m = y.size
        if m <= 1:
            return None, None
        num_parent = [np.sum(y == k) for k in range(self.n_classes)]
        best_gini = 1.0 - sum((n / m) ** 2 for n in num_parent)
        best_index, best_thr = None, None
        for index in range(self.n_features):
            thresholds, classes = zip(*sorted(zip(X[:, index], y)))
            num_left = [0] * self.n_classes    # assume left part contain zero samples
            num_right = num_parent.copy()       # assume right part contain all samples
            for i in range(1, m):
                k = classes[i - 1]
                num_left[k] += 1            # increase one in left part 
                num_right[k] -= 1           # decrease one in right part
                # finding gini 
                gini_left = 1.0 - sum((num_left[x] / i) ** 2 for x in range(self.n_classes))
                gini_right = 1.0 - sum((num_right[x] / (m - i)) ** 2 for x in range(self.n_classes))
                gini = (i * gini_left + (m - i) * gini_right) / m
                # if threshold of current sample and previous sample is same then skip
                if thresholds[i] == thresholds[i - 1]:            
                    continue
                # if current gini is less than best gini
                if gini < best_gini:
                    best_gini = gini    # replace the best gini with current gini value
                    best_index = index      # replace the feature index with current feature index
                    best_thr = (thresholds[i] + thresholds[i - 1]) / 2
        return best_index, best_thr

    

    def predict(self, inputs):
        node = self.tree
        while node.left:
            if inputs[node.feat_index] < node.threshold:
                node = node.left      # moving in left node if input is less than the threshold
            else:
                node = node.right
        return node.pred_class

    def pred(self, X):
        return [self.predict(inputs) for inputs in X] 
    

In [13]:
dataset = datasets.load_iris()
X, y = dataset.data, dataset.target  
clf = DTC(max_depth=5)          # define the tree with given depth
clf.fit(X, y)                   # train the model
input = [0, 0, 5.0, 1.5]    
pred = clf.pred([input])[0]      # predicting the class for given input
print("Input: {}".format(input))
print("Prediction: " + dataset.target_names[pred])

Input: [0, 0, 5.0, 1.5]
Prediction: virginica
